In [1]:
from dateutil.parser import parse
import email
import mailbox
import os
import re
import sys
from time import asctime

In [2]:
import pandas as pd

In [3]:
DIRNAME = os.path.dirname(os.getcwd())+os.sep
# Where to write the converted mbox
MBOX = DIRNAME+'enron'+os.sep+'enron_mail'+os.sep+'enron.mbox'

In [4]:
def maildir_to_mailbox(MAILDIR):
    with open(MBOX, 'w') as mbox:   
        for (root, dirs, file_names) in os.walk(MAILDIR):
            if root.split(os.sep)[-1].lower() != 'inbox':
                continue

            # Process each message in 'inbox'
            for file_name in file_names:
                file_path = os.path.join(root, file_name)
                message_text = open(file_path).read()

                # Compute fields for the From_ line in a traditional mbox message

                #_from = re.search(r"From: ([^\r]+)", message_text).groups()[0]
                #_date = re.search(r"Date: ([^\r]+)", message_text).groups()[0]

                # Convert _date to the asctime representation for the From_ line

                #_date = asctime(parse(_date).timetuple())

                msg = email.message_from_string(message_text)
                #msg.set_unixfrom('From %s %s' % (_from, _date))

                mbox.write(msg.as_string(unixfrom=True) + "\n\n")

    return mbox


In [5]:
maildir_to_mailbox(DIRNAME+'enron'+os.sep+'enron_mail'+os.sep+'maildir'+os.sep)

<_io.TextIOWrapper name='c:\\Users\\utilisateur\\Desktop\\Projets\\enron\\enron_mail\\enron.mbox' mode='w' encoding='cp1252'>

In [6]:
def mbox_to_df():
    """Convert the mailbox to a dataframe.

    Returns:
        dict: Enron corpus dataframe

    """
    mbox = mailbox.mbox(MBOX)

    mbox_dict = {}
    for i, msg in enumerate(mbox):
        mbox_dict[i] = {}
        for header in msg.keys():
            mbox_dict[i][header] = msg[header]
        mbox_dict[i]['Body'] = msg.get_payload().replace('\n', ' ').replace('\t', ' ').strip()

    df = pd.DataFrame.from_dict(mbox_dict, orient='index')
    df.index = df['Date'].apply(pd.to_datetime)

    return df

In [7]:
mbox_to_df()

,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,...,X-FileName,Body,Cc,Bcc,http,Re,dal,"""Notice",X-Mailer,<http
Date,,,,,,,,,,,,,,,,,,,,,
2001-12-30 22:49:42-08:00,<14955894.1075855377681.JavaMail.evans@thyme>,"Sun, 30 Dec 2001 22:49:42 -0800 (PST)",anchordesk_daily@anchordesk.zdlists.com,pallen@enron.com,ANCHORDESK: Hope ahead: What I learned from 20...,1.0,text/plain; charset=ANSI_X3.4-1968,7bit,"""AnchorDesk"" <AnchorDesk_Daily@anchordesk.zdli...",pallen@ENRON.COM,...,pallen (Non-Privileged).pst,_____________________DAVID COURSEY____________...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-12-30 23:42:30-08:00,<7462038.1075855377703.JavaMail.evans@thyme>,"Sun, 30 Dec 2001 23:42:30 -0800 (PST)",subscriptions@intelligencepress.com,pallen@enron.com,"NGI Publications - Monday, December 31st 2001",1.0,text/plain; charset=us-ascii,7bit,subscriptions@intelligencepress.com@ENRON,pallen@enron.com,...,pallen (Non-Privileged).pst,"Dear phillip, This e-mail is automated notif...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-12-31 02:24:51-08:00,<21572157.1075855377726.JavaMail.evans@thyme>,"Mon, 31 Dec 2001 02:24:51 -0800 (PST)",prizemachine@feedback.iwon.com,pallen@enron.com,"Click. Spin. Chances to Win up to $10,000!",1.0,text/plain; charset=us-ascii,7bit,Prize Machine<PrizeMachine@feedback.iwon.com>@...,pallen@enron.com,...,pallen (Non-Privileged).pst,"[IMAGE] [IMAGE] [IMAGE] [IMAGE] $ 2,500 [I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-12-31 10:53:43-08:00,<7618763.1075855377753.JavaMail.evans@thyme>,"Mon, 31 Dec 2001 10:53:43 -0800 (PST)",louise.kitchen@enron.com,"wes.colwell@enron.com, georgeanne.hodges@enron...",NETCO,1.0,text/plain; charset=us-ascii,7bit,"Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/...","Colwell, Wes </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,pallen (Non-Privileged).pst,The New Year has arrived and we really to fina...,john.lavorato@enron.com,john.lavorato@enron.com,NaN,NaN,NaN,NaN,NaN,NaN
2001-12-31 17:18:31-08:00,<4845954.1075855377776.JavaMail.evans@thyme>,"Mon, 31 Dec 2001 17:18:31 -0800 (PST)",arsystem@mailman.enron.com,k..allen@enron.com,Your Approval is Overdue: Access Request for m...,1.0,text/plain; charset=us-ascii,7bit,ARSystem <ARSystem@mailman.enron.com>@ENRON,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...",...,pallen (Non-Privileged).pst,This request has been pending your approval fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<http://www.marketwatch.com/news/story.asp?sit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,NaN,NaN,//remoteservices.netco.enron.com/ica/ubswenerg...,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,STACK MANAGER User ID: azipper Password: q#9M#...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
